In [2]:
import rvnewop as rv
from glob import glob
import matplotlib.pyplot as plt
from itertools import accumulate

In [3]:
%matplotlib inline

print("# of instructions added,{}".format(",".join(map(lambda x:str(x+1), range(150)))))

floating_point_hsts = list(map(lambda x: "./embench_hst/"+x, ["st.hst", "edn.hst", "minver.hst", "cubic.hst", "wikisort.hst", "nbody.hst"]))
for FILENAME in [x for x in glob("./embench_hst/*.hst") if x not in floating_point_hsts]:

    prog = rv.Histogram.parse(FILENAME, isa="32ICM")
    prog.findBasicBlocks()

    prog.addLivenessValuesToGraph()

    # total cycles = sum of all the frequencies
    total_cycles = sum(
        [inst.freq for bb in prog.basicBlocks for inst in bb.bbInstructions()]
    )

    new_instructions = []

    for i, b in enumerate((prog.getSubBlocks())):
        graph = b.constructDAG()

        # temp will store every permutation of candidate subgraphs
        temp = []
        for n in rv.analysis.findCandidateSubgraphs(prog, graph):
            subtree = rv.analysis.createSubtreeFromNode(graph, n)
            root = n

            temp.append(rv.Subgraph(subtree, root))

        # add candidate subgraph with highest score to new instructions list
        current_nodes = set()
        temp = sorted(temp, key=lambda sg: sg.score, reverse=True)
        while temp:
            if any([(n in current_nodes) for n in temp[0].graph.nodes]):
                temp.pop(0)
            else:
                current_nodes.update(temp[0].graph.nodes)
                new_instructions.append(temp.pop(0))

    new_instructions = sorted(new_instructions, key=lambda sg: sg.score, reverse=True)

    inst_dict = {}
    for subgraph in new_instructions:
        key = rv.analysis.graphToParenString(subgraph.graph)

        if key in inst_dict:
            inst_dict[key].score += subgraph.score
        else:
            inst_dict[key] = subgraph

    new_instructions = [inst_dict[key] for key in inst_dict]

    max_depth = 0
    max_mult_depth = 0
    for sg in new_instructions:
        if sg.containsMultiplyInstruction():
            max_mult_depth = max(max_mult_depth, sg.depth)
        else:
            max_depth = max(max_depth, sg.depth)

    saved_cycles = sum([sg.score for sg in new_instructions])
    percent_cycles = float(saved_cycles) / total_cycles * 100
    #plt.clf()
    #plt.plot([x for x in range(len(new_instructions))],list(map(lambda x:x/total_cycles, accumulate([sg.score for sg in new_instructions]))))
    print("{},{}".format(FILENAME[len("./embench_hst/"):], ",".join((map(lambda x:str(x/total_cycles), accumulate([sg.score for sg in new_instructions]))))))
    #plt.show()
    #print(
    #    """Saved {} cycles out of {} ({:.4}%) with {} new instructions""".format(
    #        saved_cycles, total_cycles, percent_cycles, len(new_instructions)
    #    )
    #)
    #print("max depth: {}\nmax mult depth: {}".format(max_depth, max_mult_depth))


# of instructions added,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150
aha-mont64.hst,0.0038763324803621035,0.004984835577635465,0.005166538662652438,0.005348241747669412,0.005529944832686385,0.0056795070418159226,0.005739289070966554,0.005790714472386452,0.005802713732717762,0.005809784725412998,0.005812641692158548,0.005813927327194045,0.005815070113892265,0.00581592720391593,0.005816784293939595,0.005817498535625983,0.0058182127773123695,0.005819498412347867,0.005820069805696978,0.005820569774877449,0.005821212592395197,0.005821783985744307,0.005